In [1]:
import torch
import tinycudann as tcnn

from torch import autograd
from torch.optim import Adam

# construct optimizables.
n_primitives = 1500
optimizables = list(range(n_primitives)) - [1, 2, 6, 7]
optm_str = ['0']*n_primitives
for idx in optimizables: optm_str[idx] = '1'
optm_str = "".join(optm_str)
print(optm_str)

# build encoding
enc_cfg = {
    "otype": "HashGrid",
    "n_levels": 1,
    "n_features_per_level": 2,
    "log2_hashmap_size": 4,
    "base_resolution": 2,
    "per_level_scale": 2.0,
    "n_primitives": 8,
    "optimizables": optm_str
}

encoding = tcnn.Encoding(
    n_input_dims=3,
    encoding_config = enc_cfg
)
print(encoding._native_tcnn_module().hyperparams())


# test optimization
num_samples = 256
x = torch.rand(num_samples, 3, device='cuda')
x = x + torch.randint(0, 8, (num_samples, 1), device="cuda") 
print('*** x: ', x.max().item(), x.min().item())

optimizer = Adam(encoding.parameters(), 2.0e-3)

print(encoding.params.clone().view(-1, 4))
for i in range(10000):
    y = encoding(x)
    loss = y.sum()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(encoding.params.clone().view(-1, 4))

01100011
{'base_resolution': 2, 'hash': 'CoherentPrime', 'interpolation': 'Linear', 'log2_hashmap_size': 1, 'n_features_per_level': 2, 'n_levels': 1, 'n_primitives': 8, 'offset': '0:2', 'optimizable': '01100011', 'otype': 'Grid', 'per_level_scale': 2.0, 'type': 'Hash'}
*** x:  7.987792491912842 0.0013281152350828052
tensor([[-7.0460e-05, -3.4031e-05,  3.8013e-05, -8.2787e-05],
        [-5.7177e-05, -4.5674e-05, -6.2585e-06,  1.4732e-05],
        [ 5.8778e-05,  4.0919e-05,  4.8354e-05,  8.2234e-05],
        [-3.9703e-05, -5.8220e-05,  6.0543e-05,  5.6743e-05],
        [ 9.4432e-05,  4.5338e-05, -7.7158e-05,  5.4780e-05],
        [ 6.6135e-05, -1.0776e-05,  9.9200e-05,  5.8157e-05],
        [ 4.1833e-05,  5.6748e-05, -3.7402e-05,  6.8916e-05],
        [ 3.8633e-05, -1.3966e-05,  2.0454e-05, -3.1430e-05]], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([[-7.0460e-05, -3.4031e-05,  3.8013e-05, -8.2787e-05],
        [-2.0000e+01, -2.0000e+01, -2.0000e+01, -2.0000e+01],
        [-2.